<a href="https://colab.research.google.com/github/WEEninja/DSL_Homework/blob/main/T04_CMOD_A7_PRNG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/pe8sutd/eda4u &> /dev/null
!git clone https://github.com/pe8sutd/eda4u &> /dev/null
%load_ext plugin

# Design steps:

# Code Breakdown

Top Module

In [ ]:
module top_module(
    input sysclk,
    input [1:0] btn,
    output pio37,
    output pio38,
    output pio39,
    output pio40,
    output pio41,
    output pio42,
    output pio43,
    output pio44,
    output pio45,
    output pio46,
    output pio47,
    output pio48,
    output [1:0] led,
    output adc_din,
    output adc_clk,
    output adc_csn,
    input  adc_dout,
    output uart_rxd_out
);

//==== RESET ====
wire rstn;
assign rstn = ~btn[1];

//==== CLOCKS ====
wire CLK500Hz, CLK1Hz, CLK_ADC, CLK_UART, CLK2Hz;

clock_div clk_div_u1(rstn, sysclk, CLK500Hz);
clock_div clk_div_u2(rstn, CLK500Hz, CLK1Hz);
clock_div clk_div_u3(rstn, sysclk, CLK_ADC);
clock_div clk_div_u4(rstn, sysclk, CLK_UART);
clock_div clk_div_u5(rstn, sysclk, CLK2Hz);

defparam clk_div_u1.FREQ_INPUT  = 12_000_000;
defparam clk_div_u1.FREQ_OUTPUT = 500;
defparam clk_div_u2.FREQ_INPUT  = 500;
defparam clk_div_u2.FREQ_OUTPUT = 1;
defparam clk_div_u3.FREQ_INPUT  = 12_000_000;
defparam clk_div_u3.FREQ_OUTPUT = 2_000_000;
defparam clk_div_u4.FREQ_INPUT  = 12_000_000;
defparam clk_div_u4.FREQ_OUTPUT = 9600;
defparam clk_div_u5.FREQ_INPUT  = 12_000_000;
defparam clk_div_u5.FREQ_OUTPUT = 2;

//==== 7-SEGMENT DISPLAY ====
reg [15:0] Segment_data;
drv_segment segment_u0(
    .rstn(rstn),
    .clk(CLK500Hz),
    .data(Segment_data),  // FIXED: pass full 16-bit random value
    .sel({pio43, pio46, pio47, pio37}),
    .seg({pio40, pio38, pio45, pio42, pio41, pio39, pio48, pio44})
);

//==== ADC CONFIG ====
localparam SINGLE_CHAN0 = 2'b10;

reg adc_ready;
wire adc_vaild;
wire [11:0] adc_data;

drv_mcp3202 drv_mcp3202_u0(
    .rstn(rstn),
    .clk(CLK_ADC),
    .ap_ready(adc_ready),
    .ap_vaild(adc_vaild),
    .mode(SINGLE_CHAN0),
    .data(adc_data),
    .port_din(adc_dout),
    .port_dout(adc_din),
    .port_clk(adc_clk),
    .port_cs(adc_csn)
);

//==== PRNG CONFIG ====
reg [15:0] rand_seed;
wire [15:0] rand_value;
reg load_seed;

prng_lfsr prng_u0(
    .clk(CLK1Hz),
    .rstn(rstn),
    .load_seed(load_seed),
    .seed(rand_seed),
    .rand_out(rand_value)
);

//==== SEED FROM ADC ====
always @(negedge rstn or posedge adc_vaild) begin
    if (!rstn) begin
        adc_ready <= 1'b0;
        Segment_data <= 16'h0000;
        rand_seed <= 16'h0000;
        load_seed <= 1'b1;
    end else if (adc_vaild) begin
        rand_seed <= {adc_data, 4'b0000};  // Extend ADC to 16-bit
        load_seed <= 1'b1;
        adc_ready <= 1'b0;
    end else begin
        adc_ready <= 1'b1;
        load_seed <= 1'b0;
    end
end

//==== UPDATE DISPLAY WITH PRNG ====
always @(posedge CLK1Hz or negedge rstn) begin
    if (!rstn)
        Segment_data <= 16'h0000;
    else
        Segment_data <= rand_value;
end

//==== UART DEBUG OUTPUT ====
reg uart_ready;
wire uart_vaild;
reg [7:0] uart_data;
reg hl_sel;

drv_uart_tx drv_uart_u0(
    .clk(CLK_UART),
    .ap_rstn(rstn),
    .ap_ready(uart_ready),
    .ap_vaild(uart_vaild),
    .tx(uart_rxd_out),
    .pairty(1'b0),
    .data(uart_data)
);

always @(negedge rstn or posedge uart_vaild or negedge CLK2Hz) begin
    if (!rstn) begin
        uart_ready <= 1'b0;
        hl_sel <= 1'b0;
    end else begin
        if (uart_vaild) begin
            uart_data <= (hl_sel) ? Segment_data[15:8] : Segment_data[7:0];
            uart_ready <= 1'b0;
        end else begin
            uart_ready <= 1'b1;
            hl_sel <= ~hl_sel;
        end
    end
end

endmodule


prng_lfsr

In [ ]:
module prng_lfsr(
    input clk,
    input rstn,
    input load_seed,
    input [15:0] seed,
    output reg [15:0] rand_out
);
    reg [15:0] lfsr;

    always @(posedge clk or negedge rstn) begin
        if (!rstn)
            lfsr <= 16'hACE1;  // fallback seed
        else if (load_seed)
            lfsr <= seed;
        else
            // taps for maximal 16-bit LFSR: 16,14,13,11
            lfsr <= {lfsr[14:0], lfsr[15] ^ lfsr[13] ^ lfsr[12] ^ lfsr[10]};
    end

    always @(*) begin
        rand_out = lfsr;
    end
endmodule


    always @(*) begin
        rand_out = lfsr;
    end

# Video

#Challenges